In [20]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps
import requests
# Import the API key.
from config import g_key

In [21]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [22]:
# Store the CSV you saved created in part one into a DataFrame.
city_weather_data_df = pd.read_csv("weather_database_files/weather_df.csv")
city_weather_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Avarua,CK,-21.21,-159.78,73.40,69,65,5.82,broken clouds,0.00,0.0
1,1,Hambantota,LK,6.12,81.12,77.00,100,77,6.93,broken clouds,0.00,0.0
2,2,Vaini,TO,-21.20,-175.20,78.80,74,40,9.17,scattered clouds,0.00,0.0
3,3,Butaritari,KI,3.07,172.79,83.71,75,93,11.54,light rain,0.11,0.0
4,4,Hualmay,PE,-11.10,-77.61,63.19,81,70,8.84,broken clouds,0.00,0.0


In [23]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
answer1 = input("Do you want it to be raining? yes or no: ") 
   
if answer1 == "yes":
    rain_ok = 1
elif answer1 == "no":
    rain_ok = 0
else:
    print("Please enter yes or no.")


answer2 = input("Do you want it to be snowing? yes or no: ")
if answer2 == "yes":
        snow_ok = 1
elif answer2 == "no":
        snow_ok = 0
else:
        print("Please enter yes or no.")
   

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? yes or no: no
Do you want it to be snowing? yes or no: no


In [24]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_weather_data_df.loc[(city_weather_data_df["Max Temp"] <= max_temp) & \
                                       (city_weather_data_df["Max Temp"] >= min_temp) & \
                                (city_weather_data_df["Rain (inches)"] >= rain_ok) & \
                                 (city_weather_data_df["Snow (inches)"] >= snow_ok)]

preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
1,1,Hambantota,LK,6.12,81.12,77.00,100,77,6.93,broken clouds,0.00,0.0
2,2,Vaini,TO,-21.20,-175.20,78.80,74,40,9.17,scattered clouds,0.00,0.0
3,3,Butaritari,KI,3.07,172.79,83.71,75,93,11.54,light rain,0.11,0.0
9,9,Bandarbeyla,SO,9.49,50.81,80.01,73,92,34.67,overcast clouds,0.00,0.0
12,12,Penugonda,IN,16.67,81.73,81.19,84,100,1.70,overcast clouds,0.00,0.0
13,13,Balaipungut,ID,1.05,101.28,75.25,97,100,2.71,light rain,0.15,0.0
18,18,Keti Bandar,PK,24.14,67.45,87.30,75,67,18.52,light rain,0.11,0.0
19,19,Bathsheba,BB,13.22,-59.52,84.20,74,40,21.92,scattered clouds,0.00,0.0
21,21,Holguin,CU,20.89,-76.26,87.80,58,20,17.22,few clouds,0.00,0.0
25,25,Thunder Bay,CA,48.40,-89.32,82.40,51,40,9.17,scattered clouds,0.00,0.0


In [25]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City","Country","Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hambantota,LK,77.00,broken clouds,6.12,81.12,
2,Vaini,TO,78.80,scattered clouds,-21.20,-175.20,
3,Butaritari,KI,83.71,light rain,3.07,172.79,
9,Bandarbeyla,SO,80.01,overcast clouds,9.49,50.81,
12,Penugonda,IN,81.19,overcast clouds,16.67,81.73,
13,Balaipungut,ID,75.25,light rain,1.05,101.28,
18,Keti Bandar,PK,87.30,light rain,24.14,67.45,
19,Bathsheba,BB,84.20,scattered clouds,13.22,-59.52,
21,Holguin,CU,87.80,few clouds,20.89,-76.26,
25,Thunder Bay,CA,82.40,scattered clouds,48.40,-89.32,


In [26]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,}

In [27]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hambantota,LK,77.00,broken clouds,6.12,81.12,Bungalow 63
2,Vaini,TO,78.80,scattered clouds,-21.20,-175.20,Keleti Beach Resort
3,Butaritari,KI,83.71,light rain,3.07,172.79,Isles Sunset Lodge
9,Bandarbeyla,SO,80.01,overcast clouds,9.49,50.81,REER XOOSHLE GROUP
12,Penugonda,IN,81.19,overcast clouds,16.67,81.73,Maruthi Residency (Lodge)


In [29]:
hotel_df.to_csv('weather_database_files/WeatherPy_vacation.csv')

In [30]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

In [31]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [32]:
# Add a map vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
hotel_df.to_csv('weather_database_files/city_choices_df.csv')